playlist_processing.ipynb

- cleaned_song_meta.json(음악 메타 데이터), train.json(플레이리스트)를 사용해 플레이리스트에 존재하는 태그들을 song_id에 매핑한다.이 때 태그는 가장 많이 나온 10개의 태그만 저장한다.
count는 플레이리스트에 얼마나 많이 등장했는지 weight를 주기 위해 저장한다. count가 10개 이상인 곡에 대해서만 저장한다.

- input: cleaned_song_meta.json, train.json
- output: processed_song.json

In [2]:
import json
import pandas as pd

"""
Method
"""

# Load cleaned song meta data(79,814)
def load_song_meta() -> pd.DataFrame:
    with open('./json_data/cleaned_song_meta.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df = df[['id', 'song_name', 'artist_name_basket']]
    df['tags'] = [[] for _ in range(len(df))]
    df['count'] = 0
    return df

# Load playlist data(115,071)
def load_playlist() -> pd.DataFrame:
    with open('./json_data/train.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)
    return df

# Add playlist's @tags to each songs and update @count to give weights.
def processing_playlist(song_df: pd.DataFrame, playlist_df: pd.DataFrame) -> pd.DataFrame:
    for idx_, row in playlist_df.iterrows():
        cnt = row.like_cnt
        song_list = row.songs
        tags = row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                song_df.at[idx, 'count'] += 1
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 10:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:10]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())
    
    for idx, row in song_df.iterrows():    
        song_df.at[idx, 'tags'] = list(dict(row.tags).keys())
    
    filter = song_df.tags.apply(lambda x: len(x) >= 10)
    song_df = song_df[filter]
    
    for idx, row in song_df.iterrows():
        song_df.at[idx, 'tags'] = ' '.join(song_df.at[idx, 'tags'])
    
    song_df = song_df[song_df['count'] >= 10]
        
    song_df.sort_values(by='count', ascending=False, inplace=True)
    return song_df
        

In [3]:
processed_song = processing_playlist(load_song_meta(), load_playlist())

In [5]:
processed_song

,id,song_name,artist_name_basket,tags,count
17382,144663,밤편지,[아이유],감성 새벽 밤 인디 새벽감성 여름밤 잠들기전 밤에듣기좋은노래 새벽에듣기좋은노래 공부,2175
14013,116573,안아줘,[정준일],새벽 감성 밤 힐링 기분전환 가을 카페 잔잔 가을감성 노래추천,2121
43453,357367,비,[폴킴],발라드 비오는날 비 밤 혼자 장마 감성발라드 편안한 센치한 감성적인,1981
44643,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],새벽 인디 밤 감성 새벽감성 잠들기전 여름밤 밤에듣기좋은노래 새벽에듣기좋은노래 공부,1919
16018,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],기분전환 힐링 카페 발라드 새벽 밤 기분좋은 인디음악 잔잔 노래추천,1633
...,...,...,...,...,...
37735,309879,Blind Faith,[넌],핑거스타일 카페 뉴에이지 기타 잔잔한 bgm 안녕하신가영 연주곡 어쿠스틱기타 신현희,10
25501,210986,안개꽃,[오이스터],인디 밤 인디입덕 노동요 카페 감각적인 인디키키 트렌디 힙합 새벽,10
67841,557791,아이셔,[홍소희],인디 부끄부끄 봄 인디음악 설레임 사랑노래 두근두근 까페 힐링 추억,10
67790,557368,XXOK,[칵스 (THE KOXX)],락 인디 기분전환 밴드 스트레스 매장음악 감성 명곡 칵스 dj딘,10


In [8]:
# save @processed_song to processed_song.json
processed_song.to_json('./json_data/processed_song.json', orient='records')